# Exploring Hackers News Posts

In this project, we'll compare two different types of posts from Hacker News, a popular site where technology related stories (or 'posts') are voted and commented upon. The two types of posts we'll explore begin with either Ask HN or Show HN.

Users submit Ask HN posts to ask the Hacker News community a specific question, such as "What is the best online course you've ever taken?" Likewise, users submit Show HN posts to show the Hacker News community a project, product, or just generally something interesting.

We'll specifically compare these two types of posts to determine the following:

Do **Ask HN** or **Show HN** receive more comments on average?
Do posts created at a certain time receive more comments on average?
It should be noted that the data set we're working with was reduced from almost **300,000** rows to approximately **20,000** rows by removing all submissions that did not receive any comments, and then randomly sampling from the remaining submissions.

## Introduction

First, we'll read in the data and remove the headers.

In [1]:
# Read in the data.
from csv import reader
opened_file = open('/Users/number1/2024/Data/Project 2/hacker_news.csv')
read_file = reader(opened_file)
hack_news = list(read_file)

#Remove Header.
hn_header = hack_news[0]
hn = hack_news[1:]
print(hn_header)
for n in hn[:5]:
    print(n)
    print('\n')

['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']
['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52']


['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30']


['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20']


['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01']


['10301696', 'Note by Note: The Making of Steinway L1037 (2007)', 'http://www.nytimes.com/2007/11/07/movies/07stein.html?_r=0', '8', '2', 'walterbell', '9/30/2015 4:12']




## Extracting Ask HN and Show HN Posts

First, we'll identify posts that begin with either **Ask HN** or **Show HN** and separate the data for those two types of posts into different lists. Separating the data makes it easier to analyze in the following steps.

In [2]:
# Identify posts that begin with either `Ask HN` or `Show HN` and separate the data into different lists.
ask_posts = []
show_posts = []
other_posts = []
for post in hn:
    post_name = post[1].lower()
    if post_name.startswith('ask hn'):
        ask_posts.append(post)
    elif post_name.startswith('show hn'):
            show_posts.append(post)
    else:
        other_posts.append(post)
print('We have',len(ask_posts), 'ask HN posts')
print('We have',len(show_posts), 'show HN posts')
print('We have', len(other_posts),'other posts')

We have 1744 ask HN posts
We have 1162 show HN posts
We have 17194 other posts


In [3]:
# Check if it's all the data
print(len(hn))
print(len(ask_posts) + len(show_posts) + len(other_posts))

20100
20100


## Calculating the Average Number of Comments for Ask HN and Show HN Posts

Now that we separated ask posts and show posts into different lists, we'll calculate the average number of comments each type of post receives.

In [4]:
#Create a function that calculate the average number
def avg(data,index):
    total = 0
    for row in data:
        value = int(row[index])
        total += value
    average = total/ len(data)
    return average     

In [5]:
#calculate the average number of comments for ask, show and other post:
avg_ask = avg(ask_posts,4)
avg_show = avg(show_posts,4)
avg_other = avg(other_posts,4)
template = 'The average number of comments for {} posts is {}'
print(template.format('ask',avg_ask))
print('\n')
print(template.format('show',avg_show))
print('\n')
print(template.format('other',avg_other))

The average number of comments for ask posts is 14.038417431192661


The average number of comments for show posts is 10.31669535283993


The average number of comments for other posts is 26.8730371059672


On average, ask posts in our sample receive approximately 14 comments, whereas show posts receive approximately 10. Since ask posts are more likely to receive comments, we'll focus our remaining analysis just on these posts.

## Finding the Amount of Ask and Show Posts and Comments by Hour Created

Next, we'll determine if we can maximize the amount of comments an ask, show post receives by creating it at a certain time. First, we'll find the amount of ask, show posts created during each hour of day, along with the number of comments those posts received. Then, we'll calculate the average amount of comments ask, show posts created at each hour of the day receive.

In [6]:
# Function to clean data(turn date and time to only hour)
import datetime as dt
def cleantime(data,index_1=4, index_2=-1):
    cleaned_time_posts = []
    for post in data:
        comments = int(post[index_1])
        date_time = post[index_2]
        date_time = dt.datetime.strptime(date_time,'%m/%d/%Y %H:%M').strftime('%H')
        cleaned_time_posts.append([comments,date_time])
    return cleaned_time_posts   

In [7]:
#Clean data:
cleaned_ask_posts = cleantime(ask_posts)
cleaned_show_posts = cleantime(show_posts)
print(cleaned_ask_posts[:3])
print('\n')
print(cleaned_show_posts[:3])

[[6, '09'], [29, '13'], [1, '10']]


[[22, '14'], [102, '22'], [1, '18']]


In [8]:
# Function to calculate the amount of ask posts created during each hour of day and the number of comments received
def calculate(data, index_1=0, index_2=1):
    dic_total = {}
    freq = {}
    for post in data:
        comments = post[index_1]
        hour = post[index_2]
        if hour in freq:
            freq[hour] += 1 
            dic_total[hour] += comments
        else:
            freq[hour] = 1
            dic_total[hour] = comments
    return dic_total, freq

In [9]:
total_ask_comments_hour, freq_ask_hour = calculate(cleaned_ask_posts)
total_show_comments_hour, freq_show_hour = calculate(cleaned_show_posts)
print(total_ask_comments_hour)
print('\n')
print(freq_ask_hour)
print('\n')
print(total_show_comments_hour)
print('\n')
print(freq_show_hour)

{'09': 251, '13': 1253, '10': 793, '14': 1416, '16': 1814, '23': 543, '12': 687, '17': 1146, '15': 4477, '21': 1745, '20': 1722, '02': 1381, '18': 1439, '03': 421, '05': 464, '19': 1188, '01': 683, '22': 479, '08': 492, '04': 337, '00': 447, '06': 397, '07': 267, '11': 641}


{'09': 45, '13': 85, '10': 59, '14': 107, '16': 108, '23': 68, '12': 73, '17': 100, '15': 116, '21': 109, '20': 80, '02': 58, '18': 109, '03': 54, '05': 46, '19': 110, '01': 60, '22': 71, '08': 48, '04': 47, '00': 55, '06': 44, '07': 34, '11': 58}


{'14': 1156, '22': 570, '18': 962, '07': 299, '20': 612, '05': 58, '16': 1084, '19': 539, '15': 632, '03': 287, '17': 911, '06': 142, '02': 127, '13': 946, '08': 165, '21': 272, '04': 247, '11': 491, '12': 720, '23': 447, '09': 291, '01': 246, '10': 297, '00': 487}


{'14': 86, '22': 46, '18': 61, '07': 26, '20': 60, '05': 19, '16': 93, '19': 55, '15': 78, '03': 27, '17': 93, '06': 16, '02': 30, '13': 99, '08': 34, '21': 47, '04': 26, '11': 44, '12': 61, '23': 36, '09'

## Calculating the Average Number of Comments for Ask HN Posts by Hour

In [10]:
# Funtion to calculate the Average
def avg(data,index_1=0, index2=1):
    avg = {}
    dic_total, freq = calculate(data)
    for hour in freq:
        avg[hour] = round(dic_total[hour]/freq[hour],2)
    return avg

In [11]:
avg_ask_posts = avg(cleaned_ask_posts)
avg_show_posts = avg(cleaned_show_posts)
print(avg_ask_posts)
print('\n')
print(avg_show_posts)

{'09': 5.58, '13': 14.74, '10': 13.44, '14': 13.23, '16': 16.8, '23': 7.99, '12': 9.41, '17': 11.46, '15': 38.59, '21': 16.01, '20': 21.52, '02': 23.81, '18': 13.2, '03': 7.8, '05': 10.09, '19': 10.8, '01': 11.38, '22': 6.75, '08': 10.25, '04': 7.17, '00': 8.13, '06': 9.02, '07': 7.85, '11': 11.05}


{'14': 13.44, '22': 12.39, '18': 15.77, '07': 11.5, '20': 10.2, '05': 3.05, '16': 11.66, '19': 9.8, '15': 8.1, '03': 10.63, '17': 9.8, '06': 8.88, '02': 4.23, '13': 9.56, '08': 4.85, '21': 5.79, '04': 9.5, '11': 11.16, '12': 11.8, '23': 12.42, '09': 9.7, '01': 8.79, '10': 8.25, '00': 15.71}


## Sorting and Printing Values from a List of Lists

In [12]:
#Function to sort and display in a following time:
def display(data):
    display_table = []
    for hour in data:
        display_table.append([hour,data[hour]])
    display_table = sorted(display_table)
    for hour in display_table:
        a_hour = dt.datetime.strptime(hour[0], "%H").strftime("%H:%M")
        avg = hour[1]
        print(a_hour,' : ', avg)

In [13]:
#Function show top 5 hours that have the largest amount of comments
def top5(data):
    display_table = []
    for hour in data:
        display_table.append([data[hour],hour])
    display_table = sorted(display_table, reverse =True)
    template ='{} : {} average comments per post'
    for hour in display_table[:5]:
        a_hour = dt.datetime.strptime(hour[1], "%H").strftime("%H:%M")
        avg = hour[0]
        print(template.format(a_hour,avg))

## Ask HN Post:

In [14]:
print('Average of comments for hour in Ask HN posts:')
display(avg_ask_posts)

Average of comments for hour in Ask HN posts:
00:00  :  8.13
01:00  :  11.38
02:00  :  23.81
03:00  :  7.8
04:00  :  7.17
05:00  :  10.09
06:00  :  9.02
07:00  :  7.85
08:00  :  10.25
09:00  :  5.58
10:00  :  13.44
11:00  :  11.05
12:00  :  9.41
13:00  :  14.74
14:00  :  13.23
15:00  :  38.59
16:00  :  16.8
17:00  :  11.46
18:00  :  13.2
19:00  :  10.8
20:00  :  21.52
21:00  :  16.01
22:00  :  6.75
23:00  :  7.99


In [15]:
print("Top 5 Hours for 'Ask HN' Comments")
top5(avg_ask_posts)

Top 5 Hours for 'Ask HN' Comments
15:00 : 38.59 average comments per post
02:00 : 23.81 average comments per post
20:00 : 21.52 average comments per post
16:00 : 16.8 average comments per post
21:00 : 16.01 average comments per post


The hour that receives the most comments per post on average is 15:00, with an average of 38.59 comments per post. There's about a 60% increase in the number of comments between the hours with the highest and second highest average number of comments.

According to the data set documentation, the timezone used is Eastern Time in the US. So, we could also write 15:00 as 3:00 pm est.

## Show HN Post:

In [17]:
print('Average of comments for hour in Show HN posts:')
display(avg_show_posts)

Average of comments for hour in Show HN posts:
00:00  :  15.71
01:00  :  8.79
02:00  :  4.23
03:00  :  10.63
04:00  :  9.5
05:00  :  3.05
06:00  :  8.88
07:00  :  11.5
08:00  :  4.85
09:00  :  9.7
10:00  :  8.25
11:00  :  11.16
12:00  :  11.8
13:00  :  9.56
14:00  :  13.44
15:00  :  8.1
16:00  :  11.66
17:00  :  9.8
18:00  :  15.77
19:00  :  9.8
20:00  :  10.2
21:00  :  5.79
22:00  :  12.39
23:00  :  12.42


In [16]:
print("Top 5 Hours for 'Show HN' Comments")
top5(avg_show_posts)

Top 5 Hours for 'Show HN' Comments
18:00 : 15.77 average comments per post
00:00 : 15.71 average comments per post
14:00 : 13.44 average comments per post
23:00 : 12.42 average comments per post
22:00 : 12.39 average comments per post


The hour that receives the most comments per post on average is 18:00, with an average of 15,77 comments per post. There's about a 52% increase in the number of comments between the hours with the highest and the average number of comments, And there isn't a big difference between 18:00 and 00:00 (15.77 - 15.71)
According to the data set documentation, the timezone used is Eastern Time in the US. So, we could also write 18:00 as 6:00 pm est.

## Conclusion

In this project, we analyzed ask, show posts and show posts to determine which type of post and time receive the most comments on average. Based on our analysis, to maximize the amount of comments a post receives, we'd recommend the post be categorized as ask post and created between 15:00 and 16:00. For show posts we'd recommend the post be created between 18:00 and 19:00, 00:00 and 01:00.